<a href="https://colab.research.google.com/github/bgalerne/M2MAS_neural_networks/blob/main/4_M2MAS_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Adversarial Networks

In [ ]:
!nvidia-smi

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torch.autograd as autograd
import os

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#GAN MNIST




Load MNIST and define some function to view images

In [ ]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
        ])

datatrain = datasets.MNIST('.', train=True, download=True,
                    transform=transform)
batch_size_train = 100
trainloader = torch.utils.data.DataLoader(datatrain,batch_size=batch_size_train,
                                           shuffle=True,num_workers=1,pin_memory=True)

def imshow(img):
    img = img/0.5 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
print('Images:')
imshow(torchvision.utils.make_grid(images, nrow=10))
# print labels
print('Labels:')
for i in range(10):
  print(' '.join('%5s' % str(labels[10*i+j].numpy()) for j in range(10)))


## Generative and discriminative models:

**Exercice**  
Define the Generator and discriminative models following the above instructions:

*Generator*  
Define a class `G_net` with the following architecture:

Generator (input : a random vector in the latent space of dimension $d$)
$d$ is a parameter of the constructor.  
The network applies the following list operations: 
1. Fully connected layer, output size 256
2. Leaky ReLU ($\alpha = 0.2$) activation
3. Fully connected layer, output size 512 
4. Leaky ReLU ($\alpha = 0.2$) activation 
5. Fully connected layer, output size 784 
6. Tanh activation
7. Reshape (1x28×28) (use `torch.view()`)

*Discriminator*  
Define a class `D_net` with the following architecture:
1. Flatten images to 1D tensors of size 784.
2. Fully connected layer, output size 512
3. Leaky ReLU ($\alpha = 0.2$) activation
4. Fully connected layer, output size 256
5. Leaky ReLU ($\alpha = 0.2$) activation
6. Fully connected layer, output size 1

Define instances of the networks using $d=32$.

**Remark:** One should add a sigmoid layer to the discriminator to output a probability but this is implicitly done in the criterion (see next exercice).


In [ ]:
# Generator network:
class G_Net(nn.Module):
  #TODO


# Discriminator network:
class D_Net(nn.Module):
  #TODO

d = 32
G_net = G_Net(d).to(device)
D_net = D_Net().to(device)




## View some generated images

In [ ]:
def show_G_net():
  with torch.no_grad():
    z = torch.randn(100,d).to(device)
    genimages = G_net(z)
    plt.figure(figsize = (5,5))
    imshow(torchvision.utils.make_grid(genimages.to('cpu'),nrow=10))
    #print(disnet(genimages))
    plt.show()

show_G_net()


## GAN training code

In PyTorch, 
the function `nn.BCEWithLogitsLoss` combines a `Sigmoid` layer and the `BCELoss`, that is,  for $(x,y)\in\mathbb{R}\times \{0,1\}$, 
$$
\ell(x,y) =  -y \cdot \log \sigma(x)
        - (1 - y) \cdot \log (1 - \sigma(x))
$$
where $\sigma: \mathbb{R}\to (0,1)$ is the sigmoid function defined by
$$
\sigma(x) = \frac{e^x}{1+e^{x}} = \frac{1}{1+e^{-x}}.
$$
The sigmoid function plays the role of the `softmax` function for binary classification since it maps $\mathbb{R}\to (0,1)$ to produce the probability of being in the class $y=1$ (and then $1 - \sigma(x)$ is the probability of being in the class $y=0$).

In the course formula of the discriminator loss, 
$$
      \max_{\theta_d}
      \underbrace{
        \frac 1 n
        \sum_{x_{\text{real}} \in \mathcal{T}_{\text{real}}} \log D_{\theta_d}(x_{\text{real}})}_{
        \substack{
          \text{force predicted labels to be 1}\\
          \text{for real images}
        }
      }
      +
      \underbrace{
        \frac1m
        \sum_{x_{\text{fake}} \in \mathcal{T}_{\text{fake}}} \log (1 - D_{\theta_d}(x_{\text{fake}}))}_{
        \substack{
          \text{force predicted labels to be 0}\\
          \text{for fake images}
        }}
$$
the sigmoid layer is implicitly included in $D_{\theta_d}$, but this will not be the case in the PyTorch implementation.
In short, 
$$
D_{\theta_d}(x) = \sigma(\mathtt{D\_{net}}(x)).
$$

**Exercice**

Implement the following training algorithm, where $b$ is the batch size (and we take $m=n=b$ in the loss formula):


> For each batch of images $x_{\text{real}}$:
> > **1) Train discriminator:**
> > > Generate $z$ a tensor of size $b\times d$ of idd Gaussian variables  
> > > Generate  $x_{\text{fake}} = \mathtt{G\_{net}}(z)$ a set $b$ fake images  
> > > Compute the (opposite of the) loss to minimize for the discriminator using `nn.BCEWithLogitsLoss`
> > > Compute the gradient and do an optimize step for the disciminator parameters  

> > **2) Train the generator:**
> > > Generate $z$ a new tensor of size $b\times d$ of idd Gaussian variables  
> > > Compute the loss to minimize
$$
      \underbrace{
        - \frac 1 m
        \sum_{z \in \mathcal T_{\text{rand}}} \log D_{\theta_d}(G_{\theta_g}(z)))
      }_{
        \substack{
          \text{force the discriminator to think that}\\
          \text{our generated fake images are real (close to 1)}
        }
      }
$$
using `nn.BCEWithLogitsLoss`  
Compute the gradient and do an optimize step for the disciminator parameters

Train the networks for 20 epochs using batch size $b=100$.
Display the current losses and show generated images at the end of each epoch.


In [ ]:

D_lr = 0.0002
G_lr = 0.0002
G_optimizer = optim.Adam(G_net.parameters(), lr=D_lr,betas=(0.5, 0.999))
D_optimizer = optim.Adam(D_net.parameters(), lr=G_lr,betas=(0.5, 0.999))

# mode collapse with SGD:
#G_optimizer = optim.SGD(G_net.parameters(), lr=lr, momentum=0.9)
#D_optimizer = optim.SGD(D_net.parameters(), lr=lr, momentum=0.9)

criterion = nn.BCEWithLogitsLoss()


In [ ]:
# TODO

## Interpolation in latent space:

**Exercice:** 

Generate 2 sets of 10 latent variable $z_0$ and $z_1$ and display the generated images by the latent variables:
$$
z_\theta = (1-\theta) z_0 + \theta z_1
$$
for $\theta$ varying between $0$ and $1$ (using the `torch.linspace` function with 20 intermediate values).
Display all the images in a grid of height 10 and width 20 images.


In [ ]:
with torch.no_grad():
  nlatent = 10
  ninterp = 20
  z0 = torch.randn(nlatent,d).to('cuda')
  z1 = torch.randn(nlatent,d).to('cuda')

  #TODO

#Sources:
Load MNIST: https://github.com/pytorch/examples/blob/master/mnist/main.py  
GAN architecture: TP GAN by Alasdair Newson: https://sites.google.com/site/alasdairnewson/teaching.  
Another more complex model: https://github.com/RAKIYOU/Training-GAN-on-MNIST